In [1]:
import numpy as np
import pandas as pd
import myutils as my

In [2]:
human = pd.read_csv('HumanActivityRecognition/train.csv')
df = human
human.shape

(7352, 563)

In [3]:
df['Activity']=df['Activity'].map({'LAYING':0,'STANDING':1,
                    'SITTING':2,'WALKING':3,
                   'WALKING_UPSTAIRS':4,'WALKING_DOWNSTAIRS':5})

In [4]:
X_train = df.drop('Activity',axis=1)
y_train = df['Activity']
X_train.shape,y_train.shape

((7352, 562), (7352,))

In [6]:
y_train.value_counts()

0    1407
1    1374
2    1286
3    1226
4    1073
5     986
Name: Activity, dtype: int64

In [7]:
X_train.shape,y_train.shape

((7352, 562), (7352,))

### pca

In [8]:
from sklearn.decomposition import PCA

pca = PCA(n_components=300)
X_train = pca.fit_transform(X_train)
X_train[:5]

array([[ 1.57272239e+01, -7.24625881e+00, -4.65589364e-01, ...,
         6.11267082e-02,  8.08641113e-02, -4.52174339e-02],
       [ 1.57233170e+01, -7.26069038e+00, -2.59002156e-01, ...,
        -5.22359390e-03, -1.73341864e-02, -3.20093588e-02],
       [ 1.57425696e+01, -7.20400561e+00,  1.06483367e-01, ...,
        -4.52057778e-03,  2.64636141e-02,  7.98452656e-03],
       [ 1.57156658e+01, -7.40380658e+00,  7.19930551e-01, ...,
         1.35068860e-02,  3.52533705e-02,  1.02663618e-02],
       [ 1.57031530e+01, -7.47370660e+00,  9.87219533e-01, ...,
         7.17468618e-03, -3.76652844e-03,  6.04387148e-03]])

In [9]:
y_train = y_train.values

In [10]:
params = [
    {
        'n_estimators' : range(30,31),
        'max_depth' : range(3,4),
        'min_samples_split' : range(2,4)
    }
]

In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()

gs = GridSearchCV(estimator=rf,
                   param_grid=params,
                  scoring='accuracy',
                  cv = 3,
                  n_jobs = -1,
                  verbose = 3)
gs.fit(X_train,y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid=[{'max_depth': range(3, 4),
                          'min_samples_split': range(2, 4),
                          'n_estimators': range(30, 31)}],
             scoring='accuracy', verbose=3)

In [12]:
gs.best_estimator_

RandomForestClassifier(max_depth=3, min_samples_split=3, n_estimators=30)

In [13]:
gs.best_score_

0.7718989611348416

In [14]:
gs.best_params_

{'max_depth': 3, 'min_samples_split': 3, 'n_estimators': 30}

In [15]:
gs.score(X_train,y_train)

0.8155603917301415

In [16]:
human_test = pd.read_csv('HumanActivityRecognition/test.csv')
human_test['Activity']=human_test['Activity'].map({'LAYING':0,'STANDING':1,
                    'SITTING':2,'WALKING':3,
                   'WALKING_UPSTAIRS':4,'WALKING_DOWNSTAIRS':5})
X_test = human_test.drop('Activity',axis=1)
y_test = human_test['Activity']

In [17]:
X_test.shape,y_test.shape

((2947, 562), (2947,))

In [18]:
X_test = pca.transform(X_test)

In [19]:
X_test.shape

(2947, 300)

In [20]:
y_pred = gs.predict(X_test)

In [21]:
my.print_score(y_test,y_pred,average='macro')

accuracy: 0.7774007465218866
precision: 0.7946530905762313
recall: 0.7946530905762313
